In [2]:
import java.util.Stack;
import java.util.function.BiFunction;

public class Token {
    private final Character token;
    private final int precedence;
    private final BiFunction<Double, Double, Double> calculation;
    private final int numArgs;

    public Token(Character token, int precedence, BiFunction<Double, Double, Double> calculation, int numArgs) {
        this.token = token;
        this.precedence = precedence;
        this.calculation = calculation;
        this.numArgs = numArgs;
    }

    public Character getToken() {
        return token;
    }

    public int getPrecedence() {
        return precedence;
    }

    public BiFunction<Double, Double, Double> getCalculation() {
        return calculation;
    }

    public int getNumArgs() {
        return numArgs;
    }
}

public class PrefixCalculator {
    private Stack<Token> operatorStack = new Stack<>();
    private Stack<Double> operandStack = new Stack<>();

    public double evaluatePrefix(String expression) {
        String[] tokens = expression.split(" ");
        // Process tokens in reverse order for prefix notation
        for (int i = tokens.length - 1; i >= 0; i--) {
            String token = tokens[i];
            if (isOperator(token)) {
                operatorStack.push(new Token(token.charAt(0), getPrecedence(token.charAt(0)), getCalculation(token.charAt(0)), 2));
            } else if (token.equals("(")) {
                // Handle grouping
                while (!operatorStack.isEmpty() && operatorStack.peek().getToken() != ')') {
                    processOperator();
                }
                operatorStack.pop(); // Remove the closing parenthesis
            } else if (token.equals(")")) {
                operatorStack.push(new Token(')', 0, null, 0)); // Push closing parenthesis
            } else {
                // Push operand onto the operand stack
                operandStack.push(Double.parseDouble(token));
            }
        }

        // Process remaining operators
        while (!operatorStack.isEmpty()) {
            processOperator();
        }

        // The final result is the only element left in the operand stack
        return operandStack.pop();
    }

    private void processOperator() {
        Token operator = operatorStack.pop();
        double operand1 = operandStack.pop();
        double operand2 = operandStack.pop();
        double result = operator.getCalculation().apply(operand1, operand2);
        operandStack.push(result);
    }

    private boolean isOperator(String token) {
        return token.equals("+") || token.equals("-") || token.equals("*") || token.equals("/");
    }

    private int getPrecedence(char operator) {
        switch (operator) {
            case '+':
            case '-':
                return 1;
            case '*':
            case '/':
                return 2;
            default:
                return 0;
        }
    }

    private BiFunction<Double, Double, Double> getCalculation(char operator) {
        switch (operator) {
            case '+':
                return (a, b) -> a + b;
            case '-':
                return (a, b) -> a - b;
            case '*':
                return (a, b) -> a * b;
            case '/':
                return (a, b) -> a / b;
            default:
                throw new IllegalArgumentException("Invalid operator: " + operator);
        }
    }

    public static void main(String[] args) {
        PrefixCalculator calculator = new PrefixCalculator();
        String expression = "* 2 - 7 5"; // Example: 2 * (7 - 5)
        double result = calculator.evaluatePrefix(expression);
        System.out.println("Result: " + result); // Output: 4.0
    }
}